In [1]:
!python3 -m pip install konlpy

### 1. Bag of Words란?
Bag of Words란 단어들의 순서는 전혀 고려하지 않고, 단어들의 출현 빈도에만 집중하는 텍스트 데이터의 수치화 표현 방법입니다.
- 갖고있는 어떤 텍스트 문서에 있는 단어들을 가방에다가 전부 넣습니다.
- 그 후에는 이 가방을 흔들어 단어들을 섞습니다.
- 만약, 해당 문서 내에서 특정 단어가 N번 등장했다면, 이 가방에는 그 특정 단어가 N개 있게됩니다.
- 또한 가방을 흔들어서 단어를 섞었기 때문에 더 이상 단어의 순서는 중요하지 않습니다.

BoW를 만드는 과정은 이렇게 두 가지 과정으로 생각해보겠습니다.
1. 각 단어에 고유한 정수 인덱스를 부여합니다. # 단어 집합 생성
2. 각 인덱스의 위치에 단어 토큰의 등장 횟수를 기록한 벡터를 만듭니다.


In [2]:
from konlpy.tag import Okt

In [3]:
okt = Okt()

In [4]:
print(okt)

In [5]:
from konlpy.tag import Okt
from collections import Counter
def build_bag_of_words(document):
    document = document.replace('.','')
    tokenized_document = okt.morphs(document)

    word_to_index = {}
    bow = []

    # 단어 빈도 계산
    word_counter = Counter(tokenized_document)
    # 단어에 인덱스 부여
    word_to_index = {word: idx for idx, word in enumerate(word_counter.keys())}

    # BoW 리스트 생성
    bow = [word_counter[word] for word in word_to_index.keys()]

    return word_to_index, bow


In [6]:
word_to_index, bow = build_bag_of_words("나는 자연어 처리를 공부하고 있다. 자연어 처리는 재미있다.")
print(f"word_to_index: {word_to_index}")
print(f"bow: {bow}")

word_to_index: {'나': 0, '는': 1, '자연어': 2, '처리': 3, '를': 4, '공부': 5, '하고': 6, '있다': 7, '재미있다': 8}
bow: [1, 2, 2, 2, 1, 1, 1, 1, 1]


### 2. CountVectorizer 클래스로 BoW 만들기
- 사이킷 런에서는 단어의 빈도를 Count하여 Vector로 만드는 CountVectorizer 클래스를 지원한다.

In [7]:
from sklearn.feature_extraction.text import CountVectorizer

corpus = ['I am studying  NLP. NLP is Fun']
vector = CountVectorizer()
print(vector.fit_transform(corpus).toarray())

print('vocabulary: ',vector.vocabulary_)

[[1 1 1 2 1]]
vocabulary:  {'am': 0, 'studying': 4, 'nlp': 3, 'is': 2, 'fun': 1}


### 3. DTM(Document-Term Matrix)??
서로 다른 문서들의 BoW들을 결합한 표현 방법입니다.

문서 단어 행렬(Document-Term Matrix, DTM)이란 다수의 문서에서 등장하는 각 단어들의 빈도를 행렬로 표현한 것을 말합니다.
쉽게 생각하면 각 문서에 대한 BoW를 하나의 행렬로 만든 것으로 생각할 수 있으며, BoW와 다른 표현 방법이 아니라 BoW 표현을 다수의 문서에 대해서 행렬로 표현하고 부르는 용어입니다.

In [8]:
from konlpy.tag import Okt
from collections import Counter

# 형태소 분석기
okt = Okt()

def build_document_term_matrix(documents):
    # 전체 단어 집합 생성
    vocabulary = set()
    tokenized_documents = []

    for document in documents:
        document = document.replace('.', '')
        tokenized = okt.morphs(document)
        tokenized_documents.append(tokenized)
        vocabulary.update(tokenized)

    # 단어 집합에 인덱스 부여
    word_to_index = {word: idx for idx, word in enumerate(sorted(vocabulary))}

    # DTM 행렬 생성
    dtm = []
    for tokenized_document in tokenized_documents:
        word_counter = Counter(tokenized_document)
        row = [word_counter.get(word, 0) for word in word_to_index.keys()]
        dtm.append(row)

    return word_to_index, dtm


In [9]:
# 문서 리스트
documents = ["먹고 싶은 사과", "먹고 싶은 바나나", "길고 노란 바나나 바나나", "저는 과일이 좋아요"]

# DTM 생성
word_to_index, dtm = build_document_term_matrix(documents)

# 단어 인덱스와 DTM 출력
print("단어 인덱스:", word_to_index)
print("DTM 행렬:")
for row in dtm:
    print(row)


단어 인덱스: {'과일': 0, '길고': 1, '노란': 2, '는': 3, '먹고': 4, '바나나': 5, '사과': 6, '싶은': 7, '이': 8, '저': 9, '좋아요': 10}
DTM 행렬:
[0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0]
[0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0]
[0, 1, 1, 0, 0, 2, 0, 0, 0, 0, 0]
[1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 1]


DTM은 매우 간단하고 구현하기도 쉽지만, 본질적으로 몇 가지 한계들이 있습니다.

1. 희소 표현(Sparse representation)

    원-핫 벡터는 단어 집합의 크기가 벡터의 차원이 되고 대부분의 값이 0이되는 벡터입니다.
    원-핫 벡터는 공간적 낭비와 계산 리소스를 증가시킬 수 있다는 점에서 단점을 가집니다.

    DTM도 마찬가지입니다.

    DTM에서 각 행을 문서 벡터라고 해봅시다.각 문서 벡터의 차원은 원-핫 벡터와 마찬가지로 전체 단어 집합의 크기를 가집니다.

    만약 가지고 있는 전체 코퍼스가 방대한 데이터라면 문서 벡터의 차원은 수만 이상의 차원을 가질수도 있습니다.

    또한 많은 문서 벡터가 대부분의 값이 0을 가질 수도 있습니다. 위 결과를 보면 알 수 있죠

    원-핫 벡터나 DTM과 같은 대부분의 값이 0인 표현을 희소 벡터(sparse vector)또는 희소 행렬(sparse matrix)라고 부르는데, 희소 벡터는 많은 양의 저장 공간과 높은 계산 복잡도를 가집니다.

    이러한 이유로 전처리를 통해 단어 집합의 크기를 줄이는 일은 BoW 표현을 사용하는 모델에서 중요할 수 있습니다.
2. 단순 빈도수 기반 접근

    여러 문서에 등장하는 모든 단어에 대해서 빈도 표기를 하는 이런 방법은 때로는 한계를 가지기도 합니다. 예를 들어 영어에 대해서 DTM을 만들었을 때, 불용어인 the는 어떤 문서이든 자주 등장할 수 밖에 없습니다. 그런데 유사한 문서인지 비교하고 싶은 문서1, 문서2, 문서3에서 동일하게 the가 빈도수가 높다고 해서 이 문서들이 유사한 문서라고 판단해서는 안 됩니다.

    각 문서에는 중요한 단어와 불필요한 단어들이 혼재되어 있습니다. 앞서 불용어(stopwords)와 같은 단어들은 빈도수가 높더라도 자연어 처리에 있어 의미를 갖지 못하는 단어라고 언급한 바 있습니다. 그렇다면 DTM에 불용어와 중요한 단어에 대해서 가중치를 줄 수 있는 방법은 없을까요? 이러한 아이디어를 적용한 TF-IDF를 이어서 학습해봅시다.

    이번에는 DTM 내에 있는 각 단어에 대한 중요도를 계산할 수 있는 TF-IDF 가중치에 대해서 알아보겠습니다. TF-IDF를 사용하면, 기존의 DTM을 사용하는 것보다 보다 많은 정보를 고려하여 문서들을 비교할 수 있습니다. TF-IDF가 DTM보다 항상 좋은 성능을 보장하는 것은 아니지만, 많은 경우에서 DTM보다 더 좋은 성능을 얻을 수 있습니다.

### 4. TF-IDF (단어 빈도 - 역 문서 빈도, Term Frequency - Inverse Document Frequency)

- TF(Term Frequency)
    - 단어의 문서 내에 출연한 횟수
    - 숫자가 클수록 문서 내에서 중요한 단어
    - 하지만, **'the'와 같은 단어도 TF값이 매우 클 것**
- IDF(Inverse Document Frequency)
    - 그 단어가 출연한 문서의 숫자의 역수
    - 값이 클수록 'the'와 같이 일반적으로 많이 쓰이는 단어

In [10]:
from math import log
# 문서 리스트
documents = ["먹고 싶은 사과", "먹고 싶은 바나나", "길고 노란 바나나 바나나", "저는 과일이 좋아요"]

vocab = sorted(list(set(w for doc in documents for w in doc.split())))
print('vocab',vocab)

def tf(w, document): # 문서 document에 w가 등장하는 횟수
    return document.count(w)
def idf(w, documents): # 단어 w가 등장하는 문서 개수의 역수
    df = 0
    for doc in documents:
        if w in doc:
            df += 1
    return log(len(documents) / (df + 1))
def tf_idf(documents):

    tf_idf_matrix = []
    for document in documents:
        tf_idf = []
        for word in vocab:
            score = tf(word, document) * idf(word, documents)
            tf_idf.append(score)
        tf_idf_matrix.append(tf_idf)

    return tf_idf_matrix


vocab ['과일이', '길고', '노란', '먹고', '바나나', '사과', '싶은', '저는', '좋아요']


In [11]:
# TF-IDF 계산
tf_idf_matrix = tf_idf(documents)

# 결과 출력
print("\nTF-IDF Matrix:")
for row in tf_idf_matrix:
    print(row)


TF-IDF Matrix:
[0.0, 0.0, 0.0, 0.28768207245178085, 0.0, 0.6931471805599453, 0.28768207245178085, 0.0, 0.0]
[0.0, 0.0, 0.0, 0.28768207245178085, 0.28768207245178085, 0.0, 0.28768207245178085, 0.0, 0.0]
[0.0, 0.6931471805599453, 0.6931471805599453, 0.0, 0.5753641449035617, 0.0, 0.0, 0.0, 0.0]
[0.6931471805599453, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.6931471805599453, 0.6931471805599453]


### DTM, TF-IDF 시각화

In [12]:
import pandas as pd

tf_idf_matrix = pd.DataFrame(tf_idf_matrix, columns = vocab)
tf_idf_matrix

,과일이,길고,노란,먹고,바나나,사과,싶은,저는,좋아요
0,0.000000,0.000000,0.000000,0.287682,0.000000,0.693147,0.287682,0.000000,0.000000
1,0.000000,0.000000,0.000000,0.287682,0.287682,0.000000,0.287682,0.000000,0.000000
2,0.000000,0.693147,0.693147,0.000000,0.575364,0.000000,0.000000,0.000000,0.000000
3,0.693147,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.693147,0.693147


In [13]:
vocab_dtm = list(word_to_index.keys())

dtm = pd.DataFrame(dtm, columns = vocab_dtm)
dtm

,과일,길고,노란,는,먹고,바나나,사과,싶은,이,저,좋아요
0,0,0,0,0,1,0,1,1,0,0,0
1,0,0,0,0,1,1,0,1,0,0,0
2,0,1,1,0,0,2,0,0,0,0,0
3,1,0,0,1,0,0,0,0,1,1,1
